In [ ]:
import os
import io
import glob
import sys
import astropy.io.fits
import matplotlib
import numpy as np
import time

import skimage.transform
import cv2
import psycopg2
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import pandas.io.sql as psql
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

from eCallistoProject import plot_config

module_path = os.path.abspath(os.path.join('radiospectra'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import radiospectra

from radiospectra.sources import CallistoSpectrogram

from matplotlib.backends.backend_pdf import PdfPages

import warnings
warnings.filterwarnings("ignore")


In [ ]:
def connect_database(dbName):
    """
    The function to create database

    Returns:
        cur  : Use the connection to get a cursor that will be used to execute queries.
        conn : The connection towards current connecting database.
    """

    # Connect to an existing database
    conn = psycopg2.connect("host=localhost port=5432 dbname={} user=postgres password=ecallistohackorange".format(dbName))

   # Set one or more parameters for the next transactions or statements in the current session.
    conn.set_session(autocommit=True)

   # Open a cursor to perform database operations
    cur = conn.cursor()

    return cur, conn


In [ ]:
cur, conn = connect_database('validation')

In [ ]:
# Databases from 2020/10/07 for Testing

def get_data_frame1():
    dataframe = psql.read_sql("SELECT * from ecallisto WHERE start_time BETWEEN '2020-10-07 12:00:00' AND '2020-10-07  15:00:00'",conn)
    return dataframe

get_data_frame1()

In [ ]:
# the average of STD(Standard deviation)

def get_std():
    Std_dataframe = psql.read_sql("SELECT instrument_name, AVG(DISTINCT std) FROM ecallisto GROUP BY instrument_name ORDER BY AVG(std)", conn)

    return Std_dataframe

get_std()

In [ ]:
# Databases for 2017/09/06

def get_data2017():
    dataframe = psql.read_sql("SELECT * from ecallisto_2017",conn)
    return dataframe

get_data2017()

In [ ]:
# Source: https://gist.github.com/salotz/8b4542d7fe9ea3e2eacc1a2eef2532c5

def move_axes(fig, ax_source, ax_target):
    old_fig = ax_source.figure
    ax_source.remove()
    ax_source.figure = fig
    ax_source.set_ylabel('')
    ax_source.set_xlabel('')
        
    ax_source.set_position(ax_target.get_position())
    ax_target.remove()
    ax_target.set_aspect("equal")
    fig.axes.append(ax_source)
    fig.add_subplot(ax_source)
    
    plt.close(old_fig)
                

In [ ]:
# calling the databases
ecallisto_DB = get_data2017()

# defining the colormap by Kushtrim 
my_colormap = matplotlib.colors.LinearSegmentedColormap.from_list("myColorMap", plot_config.COLORMAP / 255)

path = 'R:\\radio\\2002-20yy_Callisto\\2017\\09\\06'

for file in ecallisto_DB["file_name"]:
    
    path_fits = os.path.join(path,file)
    spec = CallistoSpectrogram.read(path_fits)

    # First column, Original plot
    fig1, axs1 = plt.subplots(1, 4, figsize=(20,5))
    ax1 = spec.plot(cmap=my_colormap, colorbar=None)
    ax1.title.set_text("Original Data")
    plt.close()

    # Second column, Constbacksub + elimwrongchannels
    spec2 = spec.subtract_bg("constbacksub", "elimwrongchannels")
    fig2 = plt.subplots(1, 4, figsize=(20,5))
    ax2 = spec2.plot(cmap=my_colormap, colorbar=None)
    ax2.title.set_text("Bg_cbs_rfi")
    plt.close()

    # Third column, subtract_bg_sliding_window
    spec3 = spec.subtract_bg("subtract_bg_sliding_window", window_width=800, affected_width=1,
                             amount=0.05, change_points=True)
    fig3 = plt.figure(figsize=(20,5))
    ax3 = spec3.plot(cmap=my_colormap, colorbar=None)
    ax3.title.set_text("Bg_sub_sliding_rfi")
    plt.close()

    # Fourth column, Histograms
    data_hist3 = spec2.data.flatten()
    data_hist4 = spec3.data.flatten()
    fig4, ax4 = plt.subplots(figsize=(20,5))
    ax4.hist(data_hist3 ,histtype='step', bins= 20, label='Bg_cbs_rfi')
    ax4.hist(data_hist4 ,histtype='step', bins= 20, label='Bg_sub_sliding_rfi')
    ax4.title.set_text("Histograms")
    plt.legend()
    plt.close()

    # Plot final plot by moving axes to the figure
    fig_target, (axA, axB, axC, axD) = plt.subplots(1, 4, figsize=(40,5))
    plt.suptitle(fig1._suptitle.get_text())

    move_axes(fig_target, ax1, axA)
    move_axes(fig_target, ax2, axB)
    move_axes(fig_target, ax3, axC)
    move_axes(fig_target, ax4, axD)
    plt.show()
                
print("Finished plotting!")